In [9]:
import os
import random
import numpy as np
import pandas as pd
import cv2

import torch
from torch.utils.data import Dataset,DataLoader
from torch import nn

import torchvision
from torchvision.models import mobilenet_v3_large
import torchvision.transforms as transforms

import sklearn
from sklearn import metrics, preprocessing
from sklearn.metrics import f1_score, confusion_matrix

In [3]:
"""랜덤 수의 Seed 고정, 재현성 보장"""
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [23]:
""" 업로드 파일이 이미지 포멧이 아닐 경우 Exception 처리 필요 """
def get_img(path, sub_path=None):
    try:
        im_bgr = cv2.imread(path)
        im_rgb = im_bgr[:, :, ::-1]
    except: ## 이미지 에러 발생 시 백지로 대체
        im_bgr = cv2.imread('')
        im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

In [ ]:
# """ 데이터 로드 전 전처리 """
# class CustomDataset(Dataset):
#     def __init__(
#         self, df, data_root, transform=transform_test):
        
#         super().__init__()
#         self.df = df.reset_index(drop=True).copy()
#         self.transform = transform
#         self.data_root = data_root
    
#     def __len__(self):
#         return self.df.shape[0]
    
#     def __getitem__(self, index: int):
          
#         path = "{}/{}".format(self.data_root[index], self.df.iloc[index]['image_id'])
#         img  = get_img(path)
        
#         if self.transform:
#             img = self.transform(image=img)['image']
            
#         return img 

In [18]:
""" Pre-trained 모델 호출 및 선언"""
class Model(nn.Module):
    def __init__(self, model_arch_str, num_classes= 2,pretrained=True):
        super(Model, self).__init__()
        self.backbone = models.mobilenet_v3_large(pretrained=pretrained) ## 모델 선언 여기 models.##(pretrained =pretrained)
        self.backbone.classifier[-1] = nn.Linear(self.backbone.classifier[-1].in_features, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        return x

In [4]:

def inference(images):
    # Move the images to the specified device (GPU or CPU)
    images = images.to(device)

    # Disable gradient calculation and enable inference mode
    with torch.no_grad():
        # Perform the forward pass
        outputs = model(images)

    # Apply softmax to obtain class probabilities
    probabilities = torch.softmax(outputs, dim=1)

    # Return the predicted class probabilities
    return probabilities

In [34]:
transform = transforms.Compose([
    transforms.Resize((260, 260)),
    
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [69]:
# RUN INFERENCE
from PIL import Image
if __name__ == '__main__':
    
    architecture = 'mobilenet_v3_large'
    model = Model(architecture, 16, pretrained=True)
    seed_everything(42)
    model_path = './models/mobilenet_v3_large.pth'
    
    #model = mobilenet_v3_large()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    input_image = './tst_img/toothcup.jpg' # Load your input images here
    #input_image = transform(get_img(input_image)) # Apply the transformation
    
    input_image = Image.open(input_image).convert('RGB')
    input_tensor = transform(input_image)
    input_batch = input_tensor.unsqueeze(0)
    #input_batch = input_batch.to(device)
    
    # Run inference
    output_probabilities = inference(input_batch)
    predicted_class_index = torch.argmax(output_probabilities, dim=1)
    print(predicted_class_index)
    
    

tensor([2], device='cuda:0')


In [71]:
class_labels  = ['10Kwalk', 'battery', 'box', 'else', 'bottle', 'handkerchief',
 'milk', 'paper', 'pet', 'plug', 'receipt', 'shopping bag', 'stairs',
 'transportation', 'trash picking', 'dishes']
# le = preprocessing.LabelEncoder()
# le.fit_transform(label_list)
# result = le.inverse_transform('0')

predicted_class = class_labels[predicted_class_index.item()]
print(predicted_class) 


box
